# Set up the environment

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# import tf
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K

# import os functions
import os
import time

import numpy as np
import matplotlib.pyplot as plt

from IPython import display

# Load the data

### Read the files

In [3]:
X_test = np.load("./EEG_data/X_test.npy")
y_test = np.load("./EEG_data/y_test.npy") - 769
person_train_valid = np.load("./EEG_data/person_train_valid.npy")
X_train_valid = np.load("./EEG_data/X_train_valid.npy")
y_train_valid = np.load("./EEG_data/y_train_valid.npy") - 769
person_test = np.load("./EEG_data/person_test.npy")

### Shape of data

In [4]:
print("training/Valid data shape: {}".format(X_train_valid.shape))       # training data of many persons
print("Test data shape: {}".format(X_test.shape))                        # test data of many persons
print("Training/Valid target shape: {}".format(y_train_valid.shape))     # training labels of many persons
print("Test target shape: {}".format(y_test.shape))                      # test labels of many persons
print("Person train/valid  shape: {}".format(person_train_valid.shape))  # which person correspond to the trail in test set
print("Person test shape: {}".format(person_test.shape))                 # which person correspond to the trail in test set

training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid  shape: (2115, 1)
Person test shape: (443, 1)


### divide dataset into training and validation

In [5]:
perm = np.random.permutation(X_train_valid.shape[0])
num_train = int(0.8 * X_train_valid.shape[0])
num_valid = X_train_valid.shape[0] - num_train
X_train =  X_train_valid[perm[0:num_train]]
y_train =  y_train_valid[perm[0:num_train]]
X_valid = X_train_valid[perm[num_train: ]]
y_valid = y_train_valid[perm[num_train: ]]


print("Training data shape: {}".format(X_train.shape))
print("Training label shape: {}".format(y_train.shape))
print("Validation data shape: {}".format(X_valid.shape))
print("Validation label shape: {}".format(y_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Test label shape: {}".format(y_test.shape))

Training data shape: (1692, 22, 1000)
Training label shape: (1692,)
Validation data shape: (423, 22, 1000)
Validation label shape: (423,)
Test data shape: (443, 22, 1000)
Test label shape: (443,)


### Preprocess data

In [6]:
def sliding_window(X_arr, y_arr, time_window=100, time_step=1, time_stride=1):
    temp_x = np.moveaxis(X_arr, 2, 0)
    temp_x = temp_x.astype(np.float32)
    buff = []
    
    num_slices = (len(temp_x)-time_window*time_step) // time_stride + 1
    
    # get time slices for data
    for i in range(num_slices):
        buff.append(temp_x[i*time_stride:i*time_stride + time_window*time_step:time_step])
        buff[i] = np.moveaxis(buff[i], 0, 2)
        # uncomment this if additional dimension is needed
        # buff[i] = buff[i].reshape(1, buff[i].shape[0], buff[i].shape[1], buff[i].shape[2])
        
    temp_x = np.concatenate(buff)
        
    # get time slice for labels
    temp_y = np.ones((X_arr.shape[0],num_slices))
    
    for i in range(len(y_arr)):
        temp_y[i] = temp_y[i] * y_arr[i]
        
    temp_y = temp_y.reshape((-1))
    
    return temp_x, temp_y

# Experiment 1: CRNN

In this experiment we make a model inspired by "Recurrent Deep Neural Networks for Real-Time Sleep Stage Classification From Single Channel EEG" by E. Bresch, 2018. We reduce kernel size of the filters because our data has less timestamps.

### Construct model

In [55]:
def construct_CRNN_model(TIME_WINDOW, dropout=0, regularizer=0):
    # input
    crnn_input = layers.Input(shape=(22, TIME_WINDOW))


    # ================================== CONV1 ================================== #

    t1 = tf.keras.layers.Permute((2, 1))(crnn_input)
    c1 = layers.Conv1D(32, 4, strides=1)(t1)
    new_size = TIME_WINDOW - 8 + 1
    
    bn1 = layers.BatchNormalization(axis=1)(c1)
    a1 = layers.Activation("relu")(bn1)
    do1 = layers.Dropout(0.5)(a1)
    maxpool1 = layers.MaxPooling1D(4)(do1)
    new_size = new_size//8 + 1
    
    # =========================================================================== #

    # ================================== CONV2 ================================== #

    c2 = layers.Conv1D(64, 4, strides=1)(maxpool1)
    new_size = new_size - 8 + 1
    
    bn2 = layers.BatchNormalization(axis=1)(c2)
    a2 = layers.Activation("relu")(bn2)
    do2 = layers.Dropout(0.5)(a2)
    maxpool2 = layers.MaxPooling1D(4)(do2)
    new_size = new_size//8 + 1
    
    # =========================================================================== #


    # ================================== LSTM ================================== #

    lstm3 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout, 
                        kernel_regularizer=keras.regularizers.l2(regularizer))(maxpool2)
    lstm4 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        recurrent_constraint=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm3)

    
    # =========================================================================== #

    f7 = layers.Flatten()(lstm4)

    # output
    crnn_output = layers.Dense(4, activation="softmax")(f7)
    
    return keras.Model(inputs = crnn_input, outputs = crnn_output)

In [34]:
crnn_model_1000 = construct_CRNN_model(1000, dropout=0.3, regularizer=0.001)
crnn_model_1000.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

In [35]:
crnn_model_1000.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 22, 1000)]        0         
_________________________________________________________________
permute_9 (Permute)          (None, 1000, 22)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 997, 32)           2848      
_________________________________________________________________
batch_normalization_8 (Batch (None, 997, 32)           3988      
_________________________________________________________________
activation_8 (Activation)    (None, 997, 32)           0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 997, 32)           0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 249, 32)           0   

### Make checkpoints

In [36]:
# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/crnn_1000',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]### Make checkpoints

### Train model

In [37]:
crnn_model_1000_loss_hist = crnn_model_1000.fit(X_train, y_train,
                                                validation_data = (X_valid, y_valid),
                                                epochs = 10,
                                                callbacks=checkpoint_callback)

Train on 1692 samples, validate on 423 samples
Epoch 1/10
1664/1692 [============================>.] - ETA: 0s - loss: 1.5707 - acc: 0.3047
Epoch 00001: val_loss improved from inf to 1.50053, saving model to ./model_checkpoints/crnn_1000
INFO:tensorflow:Assets written to: ./model_checkpoints/crnn_1000\assets
1692/1692 [==============================] - 14s 8ms/sample - loss: 1.5689 - acc: 0.3073 - val_loss: 1.5005 - val_acc: 0.3617
Epoch 2/10
1664/1692 [============================>.] - ETA: 0s - loss: 1.4721 - acc: 0.3552
Epoch 00002: val_loss improved from 1.50053 to 1.44891, saving model to ./model_checkpoints/crnn_1000
INFO:tensorflow:Assets written to: ./model_checkpoints/crnn_1000\assets
1692/1692 [==============================] - 12s 7ms/sample - loss: 1.4718 - acc: 0.3546 - val_loss: 1.4489 - val_acc: 0.3806
Epoch 3/10
1664/1692 [============================>.] - ETA: 0s - loss: 1.4006 - acc: 0.4159
Epoch 00003: val_loss improved from 1.44891 to 1.35931, saving model to ./mode

### Log

crnn_model_1000 = construct_CRNN_model(1000, dropout=0.3, regularizer=0.001) 55 % at best

added activity regularizer.

aug_crnn_model_1000 = construct_CRNN_model(1000, dropout=0.3, regularizer=0.01) 43% at best

aug_crnn_model_1000 = construct_CRNN_model(1000, dropout=0.5, regularizer=0.005) 53% at best

changed relu to elu, addded a new dense elu layer in the end, addded new lstm layer

construct_aug_CRNN_model(1000, dropout=0.3, regularizer=0.005)

# Experiment 2: augmented CRNN

In this experiment we add a dense layer in the end. Intuitively, it should help to form better features out of LSTM output. We also add one more layer for improved resolution.

### Construct model

In [111]:
def construct_aug_CRNN_model(TIME_WINDOW, dropout=0, regularizer=0):
    # input
    crnn_input = layers.Input(shape=(22, TIME_WINDOW))


    # ================================== CONV1 ================================== #

    t1 = tf.keras.layers.Permute((2, 1))(crnn_input)
    c1 = layers.Conv1D(32, 4, strides=1)(t1)
    new_size = TIME_WINDOW - 8 + 1
    
    bn1 = layers.BatchNormalization(axis=1)(c1)
    a1 = layers.Activation("elu")(bn1)
    do1 = layers.Dropout(0.5)(a1)
    maxpool1 = layers.MaxPooling1D(4)(do1)
    new_size = new_size//8 + 1
    
    # =========================================================================== #

    # ================================== CONV2 ================================== #

    c2 = layers.Conv1D(64, 4, strides=1)(maxpool1)
    new_size = new_size - 8 + 1
    
    bn2 = layers.BatchNormalization(axis=1)(c2)
    a2 = layers.Activation("elu")(bn2)
    do2 = layers.Dropout(0.5)(a2)
    maxpool2 = layers.MaxPooling1D(4)(do2)
    new_size = new_size//8 + 1
    
    # =========================================================================== #


    # ================================== LSTM ================================== #

    lstm3 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout, 
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(maxpool2)
    lstm4 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm3)
    
    lstm5 = layers.LSTM(16, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm4)


    
    # =========================================================================== #

    f7 = layers.Flatten()(lstm5)
    
    elu7 = layers.Dense(64 * new_size, activation="elu", kernel_regularizer=keras.regularizers.l2(regularizer))(f7)

    # output
    crnn_output = layers.Dense(4, activation="softmax")(elu7)
    
    return keras.Model(inputs = crnn_input, outputs = crnn_output)

In [115]:
aug_crnn_model_1000.compile(keras.optimizers.Adam(), "sparse_categorical_crossentropy", metrics=["acc"])
aug_crnn_model_1000 = construct_aug_CRNN_model(1000, dropout=0.3, regularizer=0.001)

In [116]:
aug_crnn_model_1000.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, 22, 1000)]        0         
_________________________________________________________________
permute_29 (Permute)         (None, 1000, 22)          0         
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 997, 32)           2848      
_________________________________________________________________
batch_normalization_48 (Batc (None, 997, 32)           3988      
_________________________________________________________________
activation_48 (Activation)   (None, 997, 32)           0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 997, 32)           0         
_________________________________________________________________
max_pooling1d_47 (MaxPooling (None, 249, 32)           0  

### Make checkpoints

In [ ]:
# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/aug_crnn_1000',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]### Make checkpoints

### Train model

In [121]:
aug_crnn_model_1000.fit(X_train, y_train,
                        validation_data = (X_valid, y_valid),
                        epochs = 10,
                        callbacks = checkpoint_callback)

Train on 1692 samples, validate on 423 samples
Epoch 1/10
1664/1692 [============================>.] - ETA: 0s - loss: 0.9223 - acc: 0.6707
Epoch 00001: val_loss did not improve from 1.10012
1692/1692 [==============================] - 26s 15ms/sample - loss: 0.9258 - acc: 0.6684 - val_loss: 1.1466 - val_acc: 0.5721
Epoch 2/10
1664/1692 [============================>.] - ETA: 0s - loss: 0.9012 - acc: 0.6863
Epoch 00002: val_loss did not improve from 1.10012
1692/1692 [==============================] - 18s 11ms/sample - loss: 0.8974 - acc: 0.6891 - val_loss: 1.1482 - val_acc: 0.5556
Epoch 3/10
1664/1692 [============================>.] - ETA: 0s - loss: 0.8814 - acc: 0.6965
Epoch 00003: val_loss did not improve from 1.10012
1692/1692 [==============================] - 18s 11ms/sample - loss: 0.8788 - acc: 0.6974 - val_loss: 1.1237 - val_acc: 0.5792
Epoch 4/10
1664/1692 [============================>.] - ETA: 0s - loss: 0.8812 - acc: 0.6845
Epoch 00004: val_loss did not improve from 1.10

# Experiment 3: optimization of augmented CRNN

In this experiment we vary parameters of CRNN to achieve better accuracy

### Construct model

In [131]:
def construct_opt_aug_CRNN_model(TIME_WINDOW,
                                 filter_size=4,
                                 dropout=0.3, 
                                 regularizer=0.001, 
                                 last_lstm_size=16, 
                                 last_hidden_layer=1000,
                                 last_dropout=0.3):
    # input
    crnn_input = layers.Input(shape=(22, TIME_WINDOW))


    # ================================== CONV1 ================================== #

    t1 = tf.keras.layers.Permute((2, 1))(crnn_input)
    c1 = layers.Conv1D(32, filter_size, strides=1)(t1)
    new_size = TIME_WINDOW - filter_size + 1
    
    bn1 = layers.BatchNormalization(axis=1)(c1)
    a1 = layers.Activation("elu")(bn1)
    do1 = layers.Dropout(0.4)(a1)
    maxpool1 = layers.MaxPooling1D(filter_size)(do1)
    new_size = new_size//filter_size + 1
    
    # =========================================================================== #

    # ================================== CONV2 ================================== #

    c2 = layers.Conv1D(64, filter_size, strides=1)(maxpool1)
    new_size = new_size - filter_size + 1
    
    bn2 = layers.BatchNormalization(axis=1)(c2)
    a2 = layers.Activation("elu")(bn2)
    do2 = layers.Dropout(0.4)(a2)
    maxpool2 = layers.MaxPooling1D(filter_size)(do2)
    new_size = new_size//filter_size + 1
    
    # =========================================================================== #


    # ================================== LSTM ================================== #

    lstm3 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout, 
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(maxpool2)
    lstm4 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm3)
    
    lstm5 = layers.LSTM(last_lstm_size, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm4)


    
    # =========================================================================== #

    f7 = layers.Flatten()(lstm5)
    
    elu7 = layers.Dense(last_hidden_layer, activation="elu", kernel_regularizer=keras.regularizers.l2(regularizer))(f7)
    do7 = layers.Dropout(last_dropout)(elu7)

    # output
    crnn_output = layers.Dense(4, activation="softmax")(do7)
    
    return keras.Model(inputs = crnn_input, outputs = crnn_output)

SyntaxError: invalid syntax (<ipython-input-131-9c95cd4297fe>, line 71)

### Vary parameters

In [130]:
TIME_WINDOW = 1000


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/opt_aug_crnn_1000_32' ,
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

opt_aug_crnn_1000 = construct_opt_aug_CRNN_model(TIME_WINDOW, 
                                                 dropout=0.3, 
                                                 regularizer=0.001, 
                                                 last_lstm_size=32, 
                                                 last_hidden_layer=1000,
                                                 last_dropout=0.3)

opt_aug_crnn_1000.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

opt_aug_crnn_1000.fit(X_train, y_train,
                      validation_data = (X_valid, y_valid),
                      epochs = 75,
                      callbacks=checkpoint_callback)

Train on 1692 samples, validate on 423 samples
Epoch 1/75
1664/1692 [============================>.] - ETA: 0s - loss: 2.9761 - acc: 0.2680
Epoch 00001: val_loss improved from inf to 2.52723, saving model to ./model_checkpoints/opt_aug_crnn_1000_32
INFO:tensorflow:Assets written to: ./model_checkpoints/opt_aug_crnn_1000_32\assets
1692/1692 [==============================] - 33s 20ms/sample - loss: 2.9697 - acc: 0.2701 - val_loss: 2.5272 - val_acc: 0.3310
Epoch 2/75
1664/1692 [============================>.] - ETA: 0s - loss: 2.2934 - acc: 0.3522
Epoch 00002: val_loss improved from 2.52723 to 2.07675, saving model to ./model_checkpoints/opt_aug_crnn_1000_32
INFO:tensorflow:Assets written to: ./model_checkpoints/opt_aug_crnn_1000_32\assets
1692/1692 [==============================] - 32s 19ms/sample - loss: 2.2898 - acc: 0.3522 - val_loss: 2.0767 - val_acc: 0.3499
Epoch 3/75
1664/1692 [============================>.] - ETA: 0s - loss: 1.9435 - acc: 0.3504
Epoch 00003: val_loss improved f

1664/1692 [============================>.] - ETA: 0s - loss: 1.0611 - acc: 0.6262
Epoch 00025: val_loss improved from 1.21499 to 1.19693, saving model to ./model_checkpoints/opt_aug_crnn_1000_32
INFO:tensorflow:Assets written to: ./model_checkpoints/opt_aug_crnn_1000_32\assets
1692/1692 [==============================] - 32s 19ms/sample - loss: 1.0605 - acc: 0.6265 - val_loss: 1.1969 - val_acc: 0.5745
Epoch 26/75
1664/1692 [============================>.] - ETA: 0s - loss: 1.0389 - acc: 0.6352
Epoch 00026: val_loss did not improve from 1.19693
1692/1692 [==============================] - 23s 14ms/sample - loss: 1.0362 - acc: 0.6377 - val_loss: 1.2746 - val_acc: 0.5390
Epoch 27/75
1664/1692 [============================>.] - ETA: 0s - loss: 1.0579 - acc: 0.6232
Epoch 00027: val_loss improved from 1.19693 to 1.12696, saving model to ./model_checkpoints/opt_aug_crnn_1000_32
INFO:tensorflow:Assets written to: ./model_checkpoints/opt_aug_crnn_1000_32\assets
1692/1692 [======================

KeyboardInterrupt: 

In [ ]:
TIME_WINDOW = 1000


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/opt_aug_crnn_1000_64' ,
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

opt_aug_crnn_1000 = construct_opt_aug_CRNN_model(TIME_WINDOW, 
                                                 dropout=0.3, 
                                                 regularizer=0.001, 
                                                 last_lstm_size=64, 
                                                 last_hidden_layer=1000,
                                                 last_dropout=0.3)

opt_aug_crnn_1000.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

opt_aug_crnn_1000.fit(X_train, y_train,
                      validation_data = (X_valid, y_valid),
                      epochs = 75,
                      callbacks=checkpoint_callback)